In [ ]:
import pandas as pd
import geopandas as gpd
from calitp_data_analysis.sql import query_sql
from calitp_data_analysis.gcs_geopandas import GCSGeoPandas
from calitp_data_analysis.geography_utils import CA_NAD83Albers_m, WGS84
gcsgp = GCSGeoPandas()
from shared_utils import gtfs_utils_v2
from shared_utils.rt_utils import show_full_df

import utils
from update_vars import ANALYSIS_DATE, shape_cols, trip_cols, stop_cols, stop_time_cols
analysis_date = ANALYSIS_DATE

# Explore Thruway Bus (and state-suppoted rail) data availability

In [ ]:
ca = gcsgp.read_parquet('gs://calitp-analytics-data/data-analyses/high_quality_transit_areas/ca_boundary.parquet')

In [ ]:
feeds = gtfs_utils_v2.schedule_daily_feed_to_gtfs_dataset_name(analysis_date)

## New SJJPA San Joaquins feed

In [ ]:
query = '''
SELECT * from cal-itp-data-infra.mart_transit_database.dim_gtfs_service_data
WHERE _is_current
LIMIT 1000
'''

In [ ]:
df = query_sql(query)

In [ ]:
sanj = df.query('name.str.contains("Amtrak San J")')

In [ ]:
sanj

In [ ]:
sanj = feeds.query('name.str.contains("Amtrak San Joaquins")')
sanj

In [ ]:
sanj_shapes = gtfs_utils_v2.get_shapes(analysis_date, operator_feeds=sanj.feed_key, shape_cols=utils.shape_cols)

In [ ]:
sanj_shapes = sanj_shapes

In [ ]:
sanj_shapes

In [ ]:
sanj_trips = gtfs_utils_v2.get_trips(analysis_date, operator_feeds=sanj.feed_key, trip_cols=utils.trip_cols)

In [ ]:
# sanj_trips.query('route_type == "2"')

In [ ]:
sanj_trips[['route_short_name', 'route_long_name', 'trip_instance_key']].groupby(['route_short_name', 'route_long_name']).count()

In [ ]:
sanj_shapes_trip_info = sanj_shapes.merge(sanj_trips.drop_duplicates(subset=['shape_array_key']), on = ['shape_array_key', 'feed_key'])

In [ ]:
# sanj_shapes_trip_info.drop(columns=['service_date']).explore(column='route_long_name')

In [ ]:
sanj_trips.query('route_short_name == "Route 99"')

## Capitol Corridor

In [ ]:
cc_feed = feeds.query('name.str.contains("Capitol")')

In [ ]:
cc_shapes = gtfs_utils_v2.get_shapes(analysis_date, operator_feeds=cc_feed.feed_key, shape_cols=utils.shape_cols)

In [ ]:
cc_trips = gtfs_utils_v2.get_trips(analysis_date, operator_feeds=cc_feed.feed_key, trip_cols=utils.trip_cols)

In [ ]:
cc_trips[['route_short_name', 'route_long_name', 'trip_instance_key']].groupby(['route_short_name', 'route_long_name']).count()

In [ ]:
cc_shapes_trip_info = cc_shapes.merge(cc_trips.drop_duplicates(subset=['shape_array_key']), on = ['shape_array_key', 'feed_key'])

In [ ]:
# cc_shapes_trip_info.drop(columns=['service_date']).explore(column='route_long_name')

## Pacific Surfliner (via Amtrak National feed)

In [ ]:
surf_feed = feeds.query('name.str.contains("Amtrak Schedule")')

In [ ]:
surf_feed

In [ ]:
surf_shapes = gtfs_utils_v2.get_shapes(analysis_date, operator_feeds=surf_feed.feed_key, shape_cols=shape_cols)

In [ ]:
surf_stops = gtfs_utils_v2.get_stops(analysis_date, operator_feeds=surf_feed.feed_key, stop_cols=stop_cols)

In [ ]:
surf_stops = surf_stops.to_crs(CA_NAD83Albers_m)
ca = ca.to_crs(CA_NAD83Albers_m)

surf_stops = surf_stops.clip(ca)

In [ ]:
surf_trips = gtfs_utils_v2.get_trips(analysis_date, operator_feeds=surf_feed.feed_key, trip_cols=trip_cols)

In [ ]:
stop_time_cols = ['feed_key', 'trip_id', 'stop_id',
                 'stop_sequence', 'arrival_time', 'departure_time',
                 'arrival_sec', 'departure_sec']

In [ ]:
surf_st = gtfs_utils_v2.get_stop_times(analysis_date, operator_feeds=surf_feed.feed_key, get_df=True,
                                      trip_df = surf_trips, stop_time_cols=stop_time_cols)

In [ ]:
surf_trips = surf_trips.query('route_long_name == "Amtrak Thruway Connecting Service"')

In [ ]:
surf_merged = surf_trips.merge(surf_st, on = ['feed_key', 'trip_id']).merge(surf_stops, on = ['feed_key', 'stop_id'])

### CA Thruway in Amtrak National Feed:

#### `route_id` 41080

* ['Chico Amtrak', 'Red Bluff Amtrak Bus Stop',
       'Redding Amtrak Bus Stop']

#### 42896

* ['Bakersfield', 'Uc Santa Barbara Amtrak Bus Stop', 'Santa Barbara',
       'Las Vegas', 'Las Vegas Amtrak Bus Stop',
       'Barstow Amtrak Bus Stop', 'Barstow', 'Mojave Amtrak Bus Stop',
       'Tehachapi Amtrak Bus Stop', 'Fillmore Amtrak Bus Stop',
       'Santa Paula Amtrak Bus Stop', 'Oxnard', 'Ventura Amtrak',
       'Carpinteria Amtrak']

#### 42954

* strange one, nationwide stops including CA points

#### `route_id` not that useful, look for trip patterns...



In [ ]:
trip_id_stops = surf_merged[['trip_id', 'stop_name']].groupby('trip_id')['stop_name'].unique().map(lambda x: frozenset(x))

In [ ]:
trip_id_stops

In [ ]:
trip_patterns = trip_id_stops.value_counts() # count unique trip patterns...

In [ ]:
trip_patterns

### Surfliner-associated Trip Patterns

https://www.pacificsurfliner.com/plan-your-trip/connections/bus-connections/

In [ ]:
sb_oakland = trip_patterns.index[2]
sb_oakland

In [ ]:
fullerton_indio = trip_patterns.index[21]

In [ ]:
fullerton_indio

In [ ]:
fullerton_palm_springs = trip_patterns.index[24]

In [ ]:
fullerton_palm_springs

In [ ]:
surfliner_routes_from_trip_patterns = {fullerton_palm_springs: 'Fullerton - Palm Springs', fullerton_indio: 'Fullerton - Indio', sb_oakland: 'Santa Barbara - Oakland'}

In [ ]:
trips_with_pattern = trip_id_stops.reset_index().rename(columns={'stop_name':'trip_pattern'})
trips_with_pattern = trips_with_pattern.assign(manual_route_name = trips_with_pattern.trip_pattern.map(
    lambda x: surfliner_routes_from_trip_patterns[x] if x in surfliner_routes_from_trip_patterns.keys() else None)
                                              )

In [ ]:
trips_with_pattern = trips_with_pattern.dropna() #  trip_id for Surfliner-associated routes...
trips_with_pattern

In [ ]:
trips_with_pattern = trips_with_pattern.merge(surf_trips, on='trip_id')

In [ ]:
trips_with_pattern

In [ ]:
#  unhelpful that these all have the same route_id!
trips_with_pattern[['route_id', 'route_long_name', 'trip_instance_key']].groupby(['route_long_name', 'route_id']).count()

In [ ]:
surf_st_merged = surf_stops.merge(surf_st, on = ['feed_key', 'stop_id']).merge(trips_with_pattern, on = ['feed_key', 'trip_id'])

## A derived timetable of Pacific Surfliner-associated thruways in the Amtrak Nationwide feed

Daily Trips:

* 1 Indio - Fullerton
* 1 Fullerton - Indio
* 1 Fullerton - Palm Springs Airport
* 1 Palm Springs Airport - Fullerton
* 4 Oakland - Santa Barbara
* 4 Santa Barbara - Oakland

In [ ]:
# show_full_df(utils.format_stop_times(surf_st_merged))

## Visualizations

In [ ]:
import chart_utils

In [ ]:
remix = gpd.read_file('Amtrak_Thruway_Bus-2025-10-15-18-37-57-route_geometry.zip').to_crs(CA_NAD83Albers_m)

In [ ]:
remix.head(1)

In [ ]:
line_17 = remix.query('line_name == "17 EMY-SBA Thruway"') #  inbound is northbound
line_17 = line_17.assign(trip_id = ['205369', '205375'])
line_17

In [ ]:
surf_st_merged.manual_route_name.unique()

In [ ]:
line_17_st = surf_st_merged.query('manual_route_name == "Santa Barbara - Oakland"')
line_17_st = line_17_st.assign(amtrak_stop = line_17_st.stop_id)

In [ ]:
#  charts only show one month for now
source_ridership = utils.read_format_ridership().query('trip_year == 2025 & trip_month == 4')

In [ ]:
chart_utils.flow_chart_from_shape_trip_row(line_17.iloc[0], stop_times=line_17_st, ridership=source_ridership,
                              ridership_data_route='Rt 17')

In [ ]:
import importlib
importlib.reload(chart_utils)

In [ ]:
chart_utils.flow_chart_from_shape_trip_row(line_17.iloc[1], stop_times=line_17_st, ridership=source_ridership,
                              ridership_data_route='Rt 17')